# <center> library </center>

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt

# <center> reading ton dataframe </center>

In [ ]:
input_path = f'your-path\\IDS-IoT\\dataset\\TON-IoT\\normal-attack\\csv-5\\ton.csv'
ton_df = pd.read_csv(input_path, on_bad_lines="error", low_memory=False)

# <center> selecting </center>

In [ ]:
X = ton_df.drop(columns=['label', 'type'])
y = ton_df['label']
z = ton_df['type']

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rfe = RFE(estimator=rf, n_features_to_select=20)
rfe = rfe.fit(X, y)

feature_ranking = rfe.ranking_
selected_features = X.columns[rfe.support_]

rf_importance = rfe.estimator_.feature_importances_

importance_min = rf_importance.min()
importance_max = rf_importance.max()
normalized_importance = (rf_importance - importance_min) / (importance_max - importance_min)

selected_features_importance = pd.DataFrame({
    'Feature': selected_features,
    'Importance': rf_importance,
    'Normalized Importance (0-1)': normalized_importance
})

selected_features_importance = selected_features_importance.sort_values(by='Importance', ascending=False)

print("\nFeature Ranking and Importance (Normalized 0-1):")
print(selected_features_importance)

plt.figure(figsize=(10, 6))
plt.barh(selected_features_importance['Feature'], selected_features_importance['Normalized Importance (0-1)'], color='skyblue')
plt.xlabel('Normalized Importance (0-1)', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance (Normalized) of Selected Features', fontsize=14)
plt.gca().invert_yaxis()  # To display the most important feature at the top
plt.tight_layout()
plt.show()

print("\nCreating DataFrame with selected features... Done.")
ton_df_selected = ton_df[selected_features].copy()

ton_df_selected['type'] = z
ton_df_selected['label'] = y
print("Added 'type' and 'label' columns back to the selected features DataFrame.")

output_path = 'your-path\\IDS-IoT\\dataset\\TON-IoT\\normal-attack\\csv-6\\ton.csv'
try:
    ton_df_selected.to_csv(output_path, index=False)
    print(f"File saved successfully to {output_path}")
except Exception as e:
    print(f"An error occurred while saving the file: {e}")
